In [ ]:
import pandas as pd
import numpy as np
import os

def merge_sector_data(mkt_cap_dir, revenue_dir):
    """
    Merge market cap and revenue data for sectors where both datasets are available.
    Handles specific CSV structures with quarterly market cap and yearly revenue data.
    """
    merged_sectors = {}
    
    # Get list of sectors from both directories
    mkt_cap_sectors = {f.split('_mkt_cap')[0] for f in os.listdir(mkt_cap_dir) if f.endswith('.csv')}
    revenue_sectors = {f.split('_revenue')[0] for f in os.listdir(revenue_dir) if f.endswith('.csv')}
    
    # Find common sectors
    common_sectors = mkt_cap_sectors.intersection(revenue_sectors)
    
    for sector in common_sectors:
        print(f"Processing sector: {sector}")
        
        # Read market cap data
        mkt_cap_file = f"{sector}_mkt_cap_quarter_end.csv"
        mkt_cap_path = os.path.join(mkt_cap_dir, mkt_cap_file)
        mkt_cap_df = pd.read_csv(mkt_cap_path)
        
        # Read revenue data
        revenue_file = f"{sector}_revenue.csv"
        revenue_path = os.path.join(revenue_dir, revenue_file)
        revenue_df = pd.read_csv(revenue_path)
        
        # Clean and convert dates
        try:
            # Handle the specific date format in market cap data
            mkt_cap_df['Date'] = pd.to_datetime(mkt_cap_df['Date'].str.split(' ').str[0])
        except AttributeError:
            # If the date is already in a different format
            mkt_cap_df['Date'] = pd.to_datetime(mkt_cap_df['Date'])
            
        revenue_df['date'] = pd.to_datetime(revenue_df['date'])
        
        # Extract year and quarter
        mkt_cap_df['year'] = pd.DatetimeIndex(mkt_cap_df['Date']).year
        mkt_cap_df['quarter'] = pd.DatetimeIndex(mkt_cap_df['Date']).quarter
        
        # Create a list to store merged data for each ticker
        merged_data = []
        
        # Get unique tickers from both datasets
        # Convert ticker columns to uppercase for consistent matching
        mkt_cap_df['Ticker'] = mkt_cap_df['Ticker'].str.upper()
        revenue_df['ticker'] = revenue_df['ticker'].str.upper()
        
        common_tickers = set(mkt_cap_df['Ticker']).intersection(set(revenue_df['ticker']))
        
        print(f"Found {len(common_tickers)} common tickers for {sector}")
        
        for ticker in common_tickers:
            # Filter data for current ticker
            ticker_mkt_cap = mkt_cap_df[mkt_cap_df['Ticker'] == ticker].copy()
            ticker_revenue = revenue_df[revenue_df['ticker'] == ticker].copy()
            
            # For each market cap entry, find the corresponding revenue
            for _, mkt_cap_row in ticker_mkt_cap.iterrows():
                matching_revenue = ticker_revenue[
                    (ticker_revenue['year'] == mkt_cap_row['year']) &
                    (ticker_revenue['quarter'] == mkt_cap_row['quarter'])
                ]
                
                if matching_revenue.empty:
                    # If no quarterly match, get the yearly revenue
                    yearly_revenue = ticker_revenue[
                        ticker_revenue['year'] == mkt_cap_row['year']
                    ]
                    
                    if not yearly_revenue.empty:
                        # Take the last available revenue entry for the year
                        revenue_value = yearly_revenue.iloc[-1]['revenue']
                        revenue_growth = yearly_revenue.iloc[-1].get('revenue_yoy_growth', np.nan)
                    else:
                        continue
                else:
                    revenue_value = matching_revenue.iloc[0]['revenue']
                    revenue_growth = matching_revenue.iloc[0].get('revenue_yoy_growth', np.nan)
                
                # Create merged row
                merged_row = {
                    'Date': mkt_cap_row['Date'],
                    'Year': mkt_cap_row['year'],
                    'Quarter': mkt_cap_row['quarter'],
                    'Ticker': ticker,
                    'MarketCap': mkt_cap_row['MarketCap'],
                    'Revenue': revenue_value,
                    'Revenue_YoY_Growth': revenue_growth,
                    'Company_Name': ticker_revenue.iloc[0].get('company_name', ticker)
                }
                merged_data.append(merged_row)
        
        if merged_data:
            # Create DataFrame from merged data
            merged_df = pd.DataFrame(merged_data)
            # Sort by Date and Ticker
            merged_df = merged_df.sort_values(['Date', 'Ticker'])
            merged_sectors[sector] = merged_df
            print(f"Successfully merged data for {sector}")
        else:
            print(f"No matching data found for {sector}")
        
    return merged_sectors



def calculate_growth_indicator(value):
    """Convert growth percentage to indicator: 1 (>5%), -1 (<-5%), 0 (between -5% and 5%)"""
    if pd.isna(value):
        return 0
    if value > 5:
        return 1
    elif value < -5:
        return -1
    else:
        return 0

def calculate_beta_covariance(df, period_months):
    """Calculate covariance of beta over specified period"""
    try:
        df = df.copy()
        
        # Calculate returns for market cap
        df['Returns'] = df.groupby('Ticker')['MarketCap'].pct_change()
        
        # Calculate market returns (using average of all companies as market proxy)
        df['Market_Returns'] = df.groupby('Date')['Returns'].transform('mean')
        
        # Calculate rolling beta for each ticker
        rolling_window = period_months * 3  # Assuming quarterly data
        
        # Initialize empty list to store betas for each ticker
        all_betas = []
        
        for ticker in df['Ticker'].unique():
            ticker_data = df[df['Ticker'] == ticker].copy()
            if len(ticker_data) > rolling_window:
                # Calculate rolling correlation
                rolling_cov = ticker_data['Returns'].rolling(window=rolling_window).cov(ticker_data['Market_Returns'])
                rolling_var = ticker_data['Market_Returns'].rolling(window=rolling_window).var()
                ticker_data['Beta'] = rolling_cov / rolling_var
                all_betas.append(ticker_data[['Date', 'Beta']].dropna())
        
        if not all_betas:
            return 0
            
        # Combine all betas
        all_betas_df = pd.concat(all_betas)
        
        # Calculate covariance matrix of betas between different dates
        beta_matrix = all_betas_df.pivot(columns='Date', values='Beta')
        cov_matrix = beta_matrix.cov()
        
        # Return average of covariance matrix (excluding diagonal)
        mask = ~np.eye(cov_matrix.shape[0], dtype=bool)
        return np.abs(cov_matrix.where(mask).mean().mean())
        
    except Exception as e:
        print(f"Error calculating beta covariance: {e}")
        return 0

def calculate_sector_rankings(merged_sectors, output_file='sector_rankings.csv'):
    """Calculate and rank sectors based on the five specified parameters"""
    rankings = []
    
    for sector, df in merged_sectors.items():
        print(f"Calculating rankings for sector: {sector}")
        try:
            # 1. Market Cap YoY Growth Indicator
            df['MktCap_YoY_Change'] = df.groupby('Ticker')['MarketCap'].pct_change(periods=4) * 100
            df['MktCap_Growth_Indicator'] = df['MktCap_YoY_Change'].apply(calculate_growth_indicator)
            
            # 2. Revenue YoY Growth Indicator
            df['Revenue_Growth_Indicator'] = df['Revenue_YoY_Growth'].apply(calculate_growth_indicator)
            
            # 3. Variance between weighted and simple average
            df['Weighted_MktCap_Change'] = (
                df['MktCap_YoY_Change'] * 
                df['MarketCap'] / 
                df.groupby('Date')['MarketCap'].transform('sum')
            )
            weighted_avg = df.groupby('Date')['Weighted_MktCap_Change'].sum().mean()
            simple_avg = df['MktCap_YoY_Change'].mean()
            variance_avg = abs(weighted_avg - simple_avg)
            
            # 4. 6-month beta covariance
            beta_6m_cov = calculate_beta_covariance(df, 2)  # 2 quarters = 6 months
            
            # 5. 5-year beta covariance
            beta_5y_cov = calculate_beta_covariance(df, 20)  # 20 quarters = 5 years
            
            rankings.append({
                'Sector': sector,
                'MktCap_Growth_Score': df['MktCap_Growth_Indicator'].mean(),
                'Revenue_Growth_Score': df['Revenue_Growth_Indicator'].mean(),
                'Weighted_Simple_Variance': variance_avg,
                'Beta_6M_Covariance': beta_6m_cov,
                'Beta_5Y_Covariance': beta_5y_cov,
                'Number_of_Companies': len(df['Ticker'].unique())
            })
            print(f"Successfully calculated rankings for {sector}")
            
        except Exception as e:
            print(f"Error processing sector {sector}: {e}")
            continue
    
    # Create rankings DataFrame and save to CSV
    rankings_df = pd.DataFrame(rankings)
    rankings_df.to_csv(output_file, index=False)
    return rankings_df

def main():
    mkt_cap_dir = "sector_mkt_cap_results"
    revenue_dir = "sector_revenue_results"
    
    # Merge sector data
    merged_sectors = merge_sector_data(mkt_cap_dir, revenue_dir)
    
    # Calculate rankings with new parameters
    rankings_df = calculate_sector_rankings(merged_sectors)
    
    # Print rankings for each parameter
    parameters = ['MktCap_Growth_Score', 'Revenue_Growth_Score', 'Weighted_Simple_Variance', 
                 'Beta_6M_Covariance', 'Beta_5Y_Covariance']
    
    for param in parameters:
        print(f"\nRanking of sectors based on {param}:")
        sorted_rankings = rankings_df.sort_values(param, ascending=False)
        for _, row in sorted_rankings.iterrows():
            print(f"{row['Sector']} ({row['Number_of_Companies']} companies): {row[param]:.4f}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
import os

def merge_sector_data(mkt_cap_dir, revenue_dir, output_dir="merged_sector_data"):
    """
    Merge market cap and revenue data for sectors where both datasets are available.
    Handles specific CSV structures with quarterly market cap and yearly revenue data.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    merged_sectors = {}
    
    # Get list of sectors from both directories
    mkt_cap_sectors = {f.split('_mkt_cap')[0] for f in os.listdir(mkt_cap_dir) if f.endswith('.csv')}
    revenue_sectors = {f.split('_revenue')[0] for f in os.listdir(revenue_dir) if f.endswith('.csv')}
    
    # Find common sectors
    common_sectors = mkt_cap_sectors.intersection(revenue_sectors)
    
    for sector in common_sectors:
        print(f"Processing sector: {sector}")
        
        # Read market cap data
        mkt_cap_file = f"{sector}_mkt_cap_quarter_end.csv"
        mkt_cap_path = os.path.join(mkt_cap_dir, mkt_cap_file)
        mkt_cap_df = pd.read_csv(mkt_cap_path)
        
        # Read revenue data
        revenue_file = f"{sector}_revenue.csv"
        revenue_path = os.path.join(revenue_dir, revenue_file)
        revenue_df = pd.read_csv(revenue_path)
        
        # Clean and convert dates
        try:
            # Handle the specific date format in market cap data
            mkt_cap_df['Date'] = pd.to_datetime(mkt_cap_df['Date'].str.split(' ').str[0])
        except AttributeError:
            mkt_cap_df['Date'] = pd.to_datetime(mkt_cap_df['Date'])
            
        revenue_df['date'] = pd.to_datetime(revenue_df['date'])
        
        # Extract year and quarter
        mkt_cap_df['year'] = pd.DatetimeIndex(mkt_cap_df['Date']).year
        mkt_cap_df['quarter'] = pd.DatetimeIndex(mkt_cap_df['Date']).quarter
        
        # Create a list to store merged data for each ticker
        merged_data = []
        
        # Get unique tickers from both datasets
        mkt_cap_df['Ticker'] = mkt_cap_df['Ticker'].str.upper()
        revenue_df['ticker'] = revenue_df['ticker'].str.upper()
        
        common_tickers = set(mkt_cap_df['Ticker']).intersection(set(revenue_df['ticker']))
        
        print(f"Found {len(common_tickers)} common tickers for {sector}")
        
        for ticker in common_tickers:
            ticker_mkt_cap = mkt_cap_df[mkt_cap_df['Ticker'] == ticker].copy()
            ticker_revenue = revenue_df[revenue_df['ticker'] == ticker].copy()
            
            for _, mkt_cap_row in ticker_mkt_cap.iterrows():
                matching_revenue = ticker_revenue[
                    (ticker_revenue['year'] == mkt_cap_row['year']) &
                    (ticker_revenue['quarter'] == mkt_cap_row['quarter'])
                ]
                
                if matching_revenue.empty:
                    yearly_revenue = ticker_revenue[
                        ticker_revenue['year'] == mkt_cap_row['year']
                    ]
                    if not yearly_revenue.empty:
                        revenue_value = yearly_revenue.iloc[-1]['revenue']
                        revenue_growth = yearly_revenue.iloc[-1].get('revenue_yoy_growth', np.nan)
                    else:
                        continue
                else:
                    revenue_value = matching_revenue.iloc[0]['revenue']
                    revenue_growth = matching_revenue.iloc[0].get('revenue_yoy_growth', np.nan)
                
                merged_row = {
                    'Date': mkt_cap_row['Date'],
                    'Year': mkt_cap_row['year'],
                    'Quarter': mkt_cap_row['quarter'],
                    'Ticker': ticker,
                    'MarketCap': mkt_cap_row['MarketCap'],
                    'Revenue': revenue_value,
                    'Revenue_YoY_Growth': revenue_growth,
                    'Company_Name': ticker_revenue.iloc[0].get('company_name', ticker)
                }
                merged_data.append(merged_row)
        
        if merged_data:
            merged_df = pd.DataFrame(merged_data)
            merged_df = merged_df.sort_values(['Date', 'Ticker'])
            merged_sectors[sector] = merged_df
            
            # Save merged data to a CSV file
            output_file_path = os.path.join(output_dir, f"{sector}_merged_data.csv")
            merged_df.to_csv(output_file_path, index=False)
            print(f"Successfully merged data for {sector} and saved to {output_file_path}")
        else:
            print(f"No matching data found for {sector}")
        
    return merged_sectors

def calculate_growth_indicator(value):
    """Convert growth percentage to indicator: 1 (>5%), -1 (<-5%), 0 (between -5% and 5%)"""
    if pd.isna(value):
        return 0
    if value > 5:
        return 1
    elif value < -5:
        return -1
    else:
        return 0

def calculate_beta_covariance(df, period_months):
    """
    Calculate covariance of beta over a specified period with improved handling of time series.
    """
    try:
        df = df.copy()
        
        # Ensure data is sorted by date
        df = df.sort_values(['Date', 'Ticker'])
        
        # Calculate returns for each company
        df['Returns'] = df.groupby('Ticker')['MarketCap'].pct_change()
        
        # Calculate market returns (using value-weighted market return)
        df['Market_Value'] = df.groupby('Date')['MarketCap'].transform('sum')
        df['Market_Weight'] = df['MarketCap'] / df['Market_Value']
        df['Market_Returns'] = df.groupby('Date')['Returns'].transform(lambda x: (x * df.loc[x.index, 'Market_Weight']).sum())
        
        # Set minimum periods for rolling calculations
        min_periods = max(2, period_months - 1)  # Ensure at least 2 periods for correlation
        rolling_window = period_months * 3  # Convert months to quarters (assuming quarterly data)
        
        betas_by_date = []
        
        for ticker in df['Ticker'].unique():
            ticker_data = df[df['Ticker'] == ticker].copy()
            
            if len(ticker_data) >= min_periods:
                # Calculate rolling betas
                rolling_cov = (
                    ticker_data['Returns']
                    .rolling(window=rolling_window, min_periods=min_periods)
                    .cov(ticker_data['Market_Returns'])
                )
                
                rolling_market_var = (
                    ticker_data['Market_Returns']
                    .rolling(window=rolling_window, min_periods=min_periods)
                    .var()
                )
                
                # To avoid dividing by zero, handle NaN or zero variance values
                ticker_data['Beta'] = rolling_cov / rolling_market_var.replace(0, np.nan)
                
                # Store results
                betas_by_date.append(ticker_data[['Date', 'Ticker', 'Beta']].dropna())
        
        if not betas_by_date:
            return 0
        
        # Combine all beta calculations
        all_betas = pd.concat(betas_by_date)
        
        # Create a pivot table of betas (companies x dates)
        beta_matrix = all_betas.pivot_table(
            index='Ticker',
            columns='Date',
            values='Beta',
            aggfunc='first'
        )
        
        # Remove companies with too many missing values
        min_observations = beta_matrix.shape[1] * 0.5  # Require at least 50% of dates
        beta_matrix = beta_matrix[beta_matrix.count(axis=1) >= min_observations]
        
        if beta_matrix.empty:
            return 0
        
        # Fill remaining NaN values with forward fill then backward fill
        beta_matrix = beta_matrix.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
        
        # Calculate covariance between different dates
        cov_matrix = beta_matrix.T.cov()
        
        # Calculate average absolute covariance (excluding diagonal)
        mask = ~np.eye(cov_matrix.shape[0], dtype=bool)
        avg_cov = np.abs(cov_matrix.where(mask)).mean().mean()
        
        return float(avg_cov) if not np.isnan(avg_cov) else 0
        
    except Exception as e:
        print(f"Error in beta covariance calculation: {e}")
        return 0

def calculate_sector_rankings(merged_sectors, output_file='sector_rankings.csv'):
    """Calculate and rank sectors based on the five specified parameters"""
    rankings = []
    
    for sector, df in merged_sectors.items():
        print(f"Processing sector: {sector}")
        try:
            # Ensure data is sorted chronologically
            df = df.sort_values('Date')
            
            # 1. Market Cap YoY Growth Indicator
            df['MktCap_YoY_Change'] = df.groupby('Ticker')['MarketCap'].pct_change(periods=4) * 100
            df['MktCap_Growth_Indicator'] = df['MktCap_YoY_Change'].apply(calculate_growth_indicator)
            
            # 2. Revenue YoY Growth Indicator
            df['Revenue_Growth_Indicator'] = df['Revenue_YoY_Growth'].apply(calculate_growth_indicator)
            
            # 3. Variance between weighted and simple average
            df['Weighted_MktCap_Change'] = (
                df['MktCap_YoY_Change'] * 
                df['MarketCap'] / 
                df.groupby('Date')['MarketCap'].transform('sum')
            )
            
            # Calculate averages only for non-NaN values
            weighted_avg = df.groupby('Date')['Weighted_MktCap_Change'].sum().mean()
            simple_avg = df['MktCap_YoY_Change'].mean()
            variance_avg = abs(weighted_avg - simple_avg)
            
            # 4 & 5. Beta covariances
            print(f"Calculating 6-month beta covariance for {sector}")
            beta_6m_cov = calculate_beta_covariance(df, 2)
            
            print(f"Calculating 5-year beta covariance for {sector}")
            beta_4y_cov = calculate_beta_covariance(df, 16)
            
            rankings.append({
                'Sector': sector,
                'MktCap_Growth_Score': df['MktCap_Growth_Indicator'].mean(),
                'Revenue_Growth_Score': df['Revenue_Growth_Indicator'].mean(),
                'Weighted_Simple_Variance': variance_avg,
                'Beta_6M_Covariance': beta_6m_cov,
                'Beta_4Y_Covariance': beta_4y_cov,
                'Number_of_Companies': len(df['Ticker'].unique()),
                'Date_Range': f"{df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}"
            })
            
            print(f"Successfully processed {sector}")
            
        except Exception as e:
            print(f"Error processing sector {sector}: {e}")
            continue
    
    # Create rankings DataFrame and save to CSV
    rankings_df = pd.DataFrame(rankings)
    rankings_df.to_csv(output_file, index=False)
    return rankings_df

def main():
    mkt_cap_dir = "sector_mkt_cap_results"
    revenue_dir = "sector_revenue_results"
    
    # Merge sector data
    merged_sectors = merge_sector_data(mkt_cap_dir, revenue_dir)
    
    # Calculate rankings with new parameters
    rankings_df = calculate_sector_rankings(merged_sectors)
    
    # Print rankings for each parameter
    parameters = ['MktCap_Growth_Score', 'Revenue_Growth_Score', 'Weighted_Simple_Variance', 
                 'Beta_6M_Covariance', 'Beta_4Y_Covariance']
    
    for param in parameters:
        print(f"\nRanking of sectors based on {param}:")
        sorted_rankings = rankings_df.sort_values(param, ascending=False)
        for _, row in sorted_rankings.iterrows():
            print(f"{row['Sector']} ({row['Number_of_Companies']} companies): {row[param]:.4f}")
            print(f"Date Range: {row['Date_Range']}")

if __name__ == "__main__":
    main()

Formulas Used in Sector Analysis:

1. Market Cap Growth Score:
   - YoY Growth % = ((Current Quarter Market Cap - Same Quarter Last Year Market Cap) / Same Quarter Last Year Market Cap) * 100
   - Score: 1 if growth > 5%, -1 if growth < -5%, 0 if -5% ≤ growth ≤ 5%
   - Final Score = Average of all company scores in sector

2. Revenue Growth Score:
   - YoY Growth % = ((Current Year Revenue - Previous Year Revenue) / Previous Year Revenue) * 100
   - Score: 1 if growth > 5%, -1 if growth < -5%, 0 if -5% ≤ growth ≤ 5%
   - Final Score = Average of all company scores in sector

3. Weighted vs Simple Average Variance:
   - Weighted Average = Σ(Company Growth * (Company Market Cap / Total Sector Market Cap))
   - Simple Average = Σ(Company Growth) / Number of Companies
   - Variance = |Weighted Average - Simple Average|

4. Beta Covariance Calculation:
   - Company Returns = (Current Market Cap - Previous Market Cap) / Previous Market Cap
   - Market Returns = Σ(Company Returns * Company Market Cap) / Total Market Cap
   - Rolling Beta = Cov(Company Returns, Market Returns) / Var(Market Returns)
   - Covariance = Average absolute covariance between different time periods' betas
"""
